# **Session 9 - Twin Delayed Deep Deterministic Policy Gradient (TD3G) Reinforecement Learning**

In [0]:
import os
import time
import random 
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque

In [0]:
# STEP 1
class ReplayBuffer(object):

  def __init__(self, max_size=1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0

  def add(self, transition):
    if len(self.storage) == self.max_size:
      self.staorage[int(self.ptr)] = transition
      self.ptr = (self.ptr + 1) % self.max_size
    else:
      self.storage.append(transition)

  def sample(self, batch_size):
    ind = np.random.randint(0, len(self.storage), batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], [], []
    for i in ind:
      state, next_state, action, reward, done = self.storage[i]
      batch_states.append(np.array(state, copy=False))
      batch_next_states.append(np.array(next_state, copy=False))
      batch_actions.append(np.array(action, copy=False))
      batch_rewards.append(np.array(reward, copy=False))
      batch_dones.append(np.array(done, copy=False)) 
    
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)

In [0]:
# STEP 2
class Actor(nn.Module):

  def __init__(self, state_dims, action_dim, max_action):
    # max action is to clip in case we added too much noise
    super(Action, self).__init__() #  activate the inheritance
    self.layer1 = nn.Linear(state_dims, 400)
    self.layer2 = nn.Linear(400, 300)
    self.layer3 = nn.Linear(300, action_dim)
    self.max_action = max_action

  def forward(self, x):
    x = F.relu(self.layer1(x))
    x = F.relu(self.layer2(x))
    x = self.max_action * torch.tanh(self.layer3(x))
    return x

In [0]:
# STEP 3
class Critic(nn.Module):

  def __init__(self, state_dims, action_dim):
    super(Critic, self).__init__()
    self.layer1 = nn.Linear(state_dims + action_dim, 400)
    self.layer2 = nn.Linear(400, 300)
    self.layer3 = nn.Linear(300, action_dim)

    self.layer4 = nn.Linear(state_dims + action_dim, 400)
    self.layer5 = nn.Linear(400, 300)
    self.layer6 = nn.Linear(300, action_dim)

  def forward(self, x, u): # x: state, u: action
    xu = torch.cat([x, u], 1) # 1 for vertical concatenation, 0 for horizontal concatenation

    # forward propagation on first Critic
    x1 = F.relu(self.layer1(xu))
    x1 = F.relu(self.layer2(x1))
    x1 = self.layer3(x1)

    # forward propagation on second Critic
    x2 = F.relu(self.layer4(xu))
    x2 = F.relu(self.layer5(x2))
    x2 = self.layer6(x2)

    return x1, x2

  def Q1(self, x, u): # x: state, u: action. This is used for updating Q values
  xu = torch.cat([x, u], 1) # 1 for vertical, 0 for horizontal concatenation
  x1 = F.relu(self.layer1(xu))
  x1 = F.relu(self.layer2(x1))
  x1 = self.layer3(x1)
  return x1 

In [0]:
# Selecting the device - CPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Building Training process into a class
class T3D(object):
  
  def __init__(self, state_dims, action_dim, max_action):
      # making sure T3D class can work with any env
      self.actor = Actor(state_dims, action_dim, max_action).to(device)
      self.actor_target = Actor(state_dims, action_dim, max_action).to(device)  # Polyak Averaging
      self.actor_target.load_state_dict(self.actor.state_dict) # initializing Target Actor with same weights as Model Actor

      self.actor_optimizer = torch.optim.Adam(self.actor.parameters())

      self.critic = Critic(state_dims, actor_dim).to(device)
      self.critic_target = Critic(state_dims, actor_dim).to(device) # Polyak Averaging
      self.critic_target.load_state_dict(self.critic.state_dict)

      self.critic_optimizer = torch.optim.Adam(self.critic.parameters())
      self.max_action = max_action

  def select_action(self, state):
    state = torch.Tensor(state.reshape(1, -1)).to(device)
    return self.actor(state).cpu().data.numpy().flatten

  def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
    # STEP 4
    for it in range(iterations):
      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      
      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)

      # STEP 5
      next_action = self.actor_target.forward(next_state)

      # STEP 6
      noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
      noise = noise.clamp(-noise_clip, noise_clip)
      next_action = (next_action + noise).clamp(-self.max_action, self.max_action)

      # STEP 7
      target_Q1, target_Q2 = self.critic_target.forward(next_state, next_action)

      # STEP 8
      target_Q = torch.min(target_Q1, target_Q2)

      # STEP 9 
      target_Q = reward + ((1 - done) * discount * target_Q).detach()

      # STEP 10
      current_Q1, current_Q2 = self.critic.forward(state, action)

      # STEP 11
      critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q2)

      # STEP 12
      self.critic_optimizer.zero_grad()
      critic_loss.backward()
      self.critic_optimizer.step()

      # STEP 13
      if it % policy_freq == 0:
        actor_loss = -(self.critic.Q1(state, self.actor(state)).mean()) # GRADIENT "ASCENT", therefore maximize. Mean Q from FIRST critic
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

      # STEP 14
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
            target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
      
      # STEP 15
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
            target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)